In [1]:
%load_ext autoreload
%autoreload 2

import warnings
warnings.filterwarnings('ignore')

import itertools

import torch
from torch import nn

import youtokentome as yttm

from utils.poetry import *
from utils.transformer_tools import *

In [2]:
def gen_some_sent(gen_model, phrase, N, beam_size, need_reweight=False, temperature=0, alpha=0):
    """
    Функция для генерации нескольких различных продолжений фразы с возможностью настройки параметров температуры и сглаживания меток
    """
    answers = []
    for i in range(N):
        beam_gen_variants = gen_model(phrase, beamsize=beam_size, return_hypotheses_n=1, need_reweight=need_reweight, temperature=temperature, alpha=alpha)
        for score, pred_txt in beam_gen_variants:
            answers.append(pred_txt)
    return answers

def test_gen(gen_model, text, temperatures, alphas):
    """
    Фукнция для перебора комбинаций температуры и сглаживания для генерации продолжения фразы
    """
    all_test = itertools.product(temperatures, alphas)
    for t, a in all_test:
        print(f"Температура = {t}, альфа = {a}")
        answers = gen_some_sent(gen_model, text, 5, 10, True, t, a)
        for ans in answers:
            print(ans)
        print("-----")

In [3]:
BPE_MODEL_FILENAME = './models/tokens/dt_bpe_2000.yttm'
tokenizer = yttm.BPE(BPE_MODEL_FILENAME)

G_model = Language_Model(
    vocab_size = tokenizer.vocab_size(),
    embedding_size = 512,
    backbone = Transformer_Encoder(
        nn.TransformerEncoderLayer(
            d_model = 512,
            nhead = 16,
            dim_feedforward = 1024,
            dropout = 0.3
        ),
        num_layers=5
    ),
    emb_dropout = 0.2
)

In [4]:
temps = [0.01, 0.2, 0.5, 0.8]
alps = [0, 0.1, 0.3, 0.5]

### Модель после генеративно-состязательной сети

In [5]:
G_model.load_state_dict(torch.load('./models/GAN/DT/2/model_G.pth'))
beam_generator = BeamGenerator(G_model, tokenizer)

In [6]:
test_gen(beam_generator, "Today we are", temps, alps)

Температура = 0.01, альфа = 0
Today we are on?” “Do you?” Mr. Bissettyouisters, Mr. Bissetteed Engineer Bob, and Charlie the Choo-Choo-Ch
Today we are." "Yes." "LAnd you?" "Yes." "YOU JAHUSTH!" Eddie shouted. "Call I?" "Yes." "Because I
Today we are of the Path of the Pubes,” Eddie said. “THISTE SUSLAMBLE MAME.” “Yes,” Blaine agreed. “It means to
Today we are.” “Drawers?” “I SUSuit,” Roland said. “I don’t know what I know.” The gunslinger nodded.“It is a great dear.” The gunslinger nodded.“It
Today we are going?” “YES,” Blaine said. “THIS PURY SALL THE MEME OF NEW YEVER INTERES RIDDLES ARE
-----
Температура = 0.01, альфа = 0.1
Today we are, Miz ASKAY OF NEW YORK. AS ARE YOU KNOWER MY ME AST ALL ARE NOTHER
Today we are playing-World RIDDLES,” Blaine said. “THAME RIDDLES OF NEWERS, ROLAND OF GUNSLINGER
Today we are Did-World, the Great Road of Grays and the Barony Copperhead?” “Ka,” the Gard said Mr. Bissette’s
Today we are,” Mr. Bury Topeka, Mr. Bississette and Mr. Charlie the Choo-Choo

In [7]:
test_gen(beam_generator, "For the King and", temps, alps)

Температура = 0.01, альфа = 0
For the King and Engineer Bob had been a Final Esayssay. Eseside Engineer Bob’s Engineer Bob said Engine Charlie
For the King and Topeka Topeka Luca Topeka Topeka Topeka and Topeka, Eurspected Engineer Bob was
For the King and Fair-Day, 19th Street of 19860, 1970, and TAMENT JAKE THREAKE TRAKE
For the King and In the Barony Coffin Hoing Below Moach of Turo’s Turtle Burtle Building Burtle B
For the King and the Mid-World Railway Coach of St. Louis Stranger Topeka St. Topeka Topeka St. Topek
-----
Температура = 0.01, альфа = 0.1
For the King and the Big Engineer Bob insurgency Bob’s English Bob. Charlie’s Choo-CK. Engineer Bob, Charlie
For the King and Edo Bocomald Star Topeka’s Evin English Topeka’s Topeka Topeka Topeka
For the King and Turtlexes of 1987, 1987197016387, Odetta 986387 96270
For the King and Buildrent Coming Buzzl Engl Engine Bob’s Bob and Charlie was Charlie the Choo-Choo, Choo-Ch
For the King and Turtlexes Topeka, Kansas Curtletle of Topeka

In [8]:
test_gen(beam_generator, "We are going to", temps, alps)

Температура = 0.01, альфа = 0
We are going to let us run out of your own mind. Final Esssay, Blaine is the Path of the Mid-World Riddle-De-DDum!-D
We are going to happen, Blaine.” “I SHABLENAH OF COULD HAVE YOUR RIDDLES, ROLAND OF GILEAD, I C
We are going to tailure me." He looked at the gunslinger. "Call I don't know." "If you." "Sure me." Eddie looked at the gunslinger with a sudden speculiation that
We are going to Cltain Roadio Cany COMMOSE. CAN HACOINED AREE,” Blaine said in a low voice. “I CLOPSE
We are going to Claudio Claudio City Miz HAMAVE OR RIDDLE, ROLAND OF GILENAMELDLE
-----
Температура = 0.01, альфа = 0.1
We are going to call the Rhe<EOS>
We are going to use the Dark Tower,” said in a low voice. “THISHE IS THE QUICALK OF GUNAY OF NOW HEREW YORK.”
We are going to kill our feet. We’re going to die, we?” Susannah asked. Roland shrugged, but Eddie only shook his head.“I don’t know what’s wrong.” Roland shook his head.“I don’t think
We are going to test.” “All right,” Roland 

In [9]:
test_gen(beam_generator, "That long story is", temps, alps)

Температура = 0.01, альфа = 0
That long story is the truth. Blaine is the truth. Blaine is the truth. Blaine is the truth. Blaine is the truth. Blaine is the truth. Blaine is the truth. Blaine is the truth. Blaine truth
That long story is the truth. Blaine is the truth, is the truth. Blaine is the truth. Blaine is the truth. Blaine is the truth. Blaine is the truth. Blaine is the truth. Blaine is the
That long story is.” “AST YOU RIDDLE ME, GOT, ROLAND OF GILEAD, Blaine, GUDLESLING OF NEW YORK
That long story is Blaine the Mon of Engineer Bob, Engineer Bob and Charlie the Choo-Choo, Charlie’s Choo. Choo-Choo,
That long story is also the truth of the truth. Blaine is the truth. Blaine is the truth. Blaine is the truth. The truth. Blaine is the truth. Blaine is the truth. Blaine is the truth.
-----
Температура = 0.01, альфа = 0.1
That long story isn’t the truth. Blaine is the truth. Blaine is the truth. Blaine is the truth. Blaine is the truth. Blaine is the truth. Blaine is the truth. B

### Модель до генеративно-состязательной сети

In [10]:
G_model.load_state_dict(torch.load('./models/Lord/DT/2/Lord.pth'))
beam_generator = BeamGenerator(G_model, tokenizer)

In [11]:
test_gen(beam_generator, "Today we are", temps, alps)

Температура = 0.01, альфа = 0
Today we are the Beast of the Turtle.” “THE IS YOUR RUDLEST, ROLAND SONE OF YOUR RIDDLES?” Blaine asked.
Today we are very well.” “I’m fine.” “Thanswer,” Roland said. “I’m sorry,” Roland said, and then ignored him. “But we were-” “Come on, let’s
Today we are we?” Susannah asked. Eddie nodded.“No, Blaine.” “THATHIS IS SHATHAT IS YOUR RIDDLE, ROLAND OF YOUR RIDD
Today we are.” 80 THIS THE SOORD THE CHAMENS OF THE PROS AREAINE OF NEW YORK. I WAS
Today we are?” “I don’t know,” the voice said. “THIS GUNSTLENS HUNSLINGER.” The voice of the voice of Little Blaine’s voice was soft
-----
Температура = 0.01, альфа = 0.1
Today we are, are you?" "Yes." "LDetta stole on my guard." "So," the gunslinger said in a low voice. "Kaka," Eddie said. "Keept
Today we are watch. Topeka, Mr. Kansas. Keflex was a cement. Kennountlex of Kansas Carl Delevan said K
Today we are going to sorrow.” “YOULD YOU LIGHTLE MY, RIDDLES, ROLAND OF GUDLESLINGER,” Blaine
Today we area?” Susannah a

In [12]:
test_gen(beam_generator, "For the King and", temps, alps)

Температура = 0.01, альфа = 0
For the King and Turtley Topeka’s Topeka and Talitha Zoltan. Topeka Topeka Zoltan.Z
For the King and Engineer Boyl Engineer Bob and Charlie the Choo-Choo, Choo-Choo-Choo. Engineer Bob
For the King and Mid-World Rails of the Grays and Fair-Days 9th Street, and the Topeka Zoltan. Top
For the King and Topeka Trid-World Mid-World Railway Company St. Louis Lana Louis Louis Reap Bad Man
For the King and Turtletle of Final Esssssayss of the Great Road, and the Cradlet Coms of Kansas,
-----
Температура = 0.01, альфа = 0.1
For the King and Topeka of Topeka Za and Topeka Topeka Zoltan and Topeka Topeka Topeka of
For the King and 7 EDDIE OF NEW YORK. ROLAND OF GILESTLEAD, I COMPLE MACE TO WITH
For the King and Ength. Engineer Bob, Charlie the Choo-Choo-Choo and Engineer Bob. Charlie the Choo-Choo.
For the King and the Glay Roys and the City of Longreland Royal Crossing of Gilead Centrays Hounds and Parks H
For the King and Triansion totle Artle of the Con Royals Curt

In [13]:
test_gen(beam_generator, "We are going to", temps, alps)

Температура = 0.01, альфа = 0
We are going to light. Topeka. Topeka. Topeka Topeka Long in Engineer Bob, and Charlie’s Choo-Choo.
We are going to be done. Pliggles, the Customs agents of the Cradle and Pubberton Roing Piper School. From
We are going to die me?” “Yes,” Roland said. “THIS TRAME OF YOUR QUILY RILEAD, ROLAND OF GILEAD IS NEW
We are going to stand on the other side.” “THISE IS NO DEVERY OR RIDDLE OF NEW YORK.” “Thankee-sai
We are going to the Dark Tower.” “Then-” “Then Katzeflex.” “All right,” Eddie said. “If all right.” Roland nodded and shook his head.“It’s a lot
-----
Температура = 0.01, альфа = 0.1
We are going to get out of here. I think we’re supposed to be here and it’s gonna be able to be a little true.” Eddie looked up at Susannah, then looked at Susannah.“It’s
We are going to have a lot of weeds. Topeka, Kansas,” Jake said Keflectly, “but I know the Kansas Clex City . .45
We are going to free.” “I AMANGHT THAT IN THE PROPEALTICROGICIONS, ROLAND.” “Thankee-s
We are

In [14]:
test_gen(beam_generator, "That long story is", temps, alps)

Температура = 0.01, альфа = 0
That long story is clear of the truth. Engineer Bob insist, and Charlie the Choo-Choo-Choo. Choo.Choo-choo. Choo,
That long story is End-World. I don’t like this SHADE. I WAS RIDDLES YOU WITH YOU RIDDLES, ROLAND OF GILE
That long story is still. Blaine is the truth. Blaine is the truth. Blaine is the truth. Blaine is Blaine is the truth. Blaine is the truth. Blaine is the truth. Blaine is the truth. Blaine
That long story is HAVE, BUT OF NEWAY HEREWARD, YOUR NEW YORK. RIDDLES, ROLAND OF GI
That long story is Blaine is the truth of the truth. Blaine is the truth. Blaine is the truths and the truth. Blaine is the truth. Blaine is the truth. Blaine is the truth. Blaine is the
-----
Температура = 0.01, альфа = 0.1
That long story is our business. Guy,” Eddie said. “Is it possible tonight, but I think it will.” Roland nodded.“Wext. I think we’ll be able to get
That long story is Blaine the truth. Blaine is the truth. The truth. Blaine is the truth. Blaine is Bl